In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
file_path = './reports/user_mwater_data.xlsx'
complete_data = pd.read_excel(file_path)

In [3]:
# Create a clean_data dataset by deduplicating based on 'user_ID'
clean_data = complete_data.drop_duplicates(subset='user_ID', keep='last')


In [4]:
clean_data.shape
complete_data.shape

(90, 18)

(171, 18)

In [ ]:
# Convert 'Last Activity' to naive datetime (without timezone) for consistency
clean_data['Last Activity'] = pd.to_datetime(clean_data['Last Activity'], errors='coerce').dt.tz_localize(None)

# Define the 12-month cutoff using naive datetime for consistency
cutoff_date = pd.Timestamp.now().tz_localize(None) - pd.DateOffset(months=12)


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_80608/4223700939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Last Activity'] = pd.to_datetime(clean_data['Last Activity'], errors='coerce').dt.tz_localize(None)


## active_users_12_months

In [ ]:
active_users_12_months = clean_data[clean_data['Last Activity'] >= cutoff_date]
num_active_users_12_months = len(active_users_12_months)
num_active_users_12_months

52

## 7 days

In [ ]:
# Total unique active users (either app or portal)
total_active_users = len(clean_data[(clean_data['Portal Activity 7 days'] > 0) | (clean_data['App Activity 7 days'] > 0)])
total_active_users
active_users = clean_data[(clean_data['Portal Activity 7 days'] > 0) | (clean_data['App Activity 7 days'] > 0)]

13

In [8]:
# Corrected calculation for active users in the portal and app (instead of summing the activity)
portal_active_users = clean_data[clean_data['Portal Activity 7 days'] > 0]
app_active_users = clean_data[clean_data['App Activity 7 days'] > 0]
total_portal_active_users = portal_active_users.shape[0]
total_app_active_users = app_active_users.shape[0]

total_portal_active_users
total_app_active_users

11

4

In [ ]:
# remove extra
app_active_users = app_active_users[~(app_active_users.user_ID.isin(portal_active_users.user_ID))]
app_active_users.shape[0]
total_app_active_users = app_active_users.shape[0]

2

In [10]:
# Calculate percentages
portal_usage_percentage = (total_portal_active_users / total_active_users) * 100 if total_active_users > 0 else 0
app_usage_percentage = (total_app_active_users / total_active_users) * 100 if total_active_users > 0 else 0

round(portal_usage_percentage)
round(app_usage_percentage)

portal_usage_percentage = round(portal_usage_percentage)

app_usage_percentage = round(app_usage_percentage)


85

15

In [ ]:
# Number of inactive users overall (users with no activity in both app and portal)
inactive_users = clean_data[(clean_data['App Activity 7 days'] == 0) & (clean_data['Portal Activity 7 days'] == 0)]
num_inactive_users = len(inactive_users)
num_inactive_users

77

In [12]:
# Exclude the 'HANWASH' column from the committee analysis
committee_columns_no_hanwash = [
    'Ambassadors', 'Champion Partnerships', 'D7020 Engagement', 'Executive', 'Finance Legal and Admin',
    'Haiti Liaison', 'MAGEPA', 'Marketing and Donor Relations', 'Monitoring and Evaluation',
    'Projects Support Subcommittee', 'Steering Committee'
]
"""
# Active and inactive users by committee (excluding HANWASH)
active_users_by_committee_no_hanwash = active_users[committee_columns_no_hanwash].apply(lambda col: (col == 'Yes').sum())
inactive_users_by_committee_no_hanwash = inactive_users[committee_columns_no_hanwash].apply(lambda col: (col == 'Yes').sum())

# Calculate the percentage of active users in each committee (excluding HANWASH)
active_users_percentage_by_committee_no_hanwash = (active_users_by_committee_no_hanwash / active_users.shape[0]) * 100
active_users_percentage_by_committee_no_hanwash """

"\n# Active and inactive users by committee (excluding HANWASH)\nactive_users_by_committee_no_hanwash = active_users[committee_columns_no_hanwash].apply(lambda col: (col == 'Yes').sum())\ninactive_users_by_committee_no_hanwash = inactive_users[committee_columns_no_hanwash].apply(lambda col: (col == 'Yes').sum())\n\n# Calculate the percentage of active users in each committee (excluding HANWASH)\nactive_users_percentage_by_committee_no_hanwash = (active_users_by_committee_no_hanwash / active_users.shape[0]) * 100\nactive_users_percentage_by_committee_no_hanwash "

In [13]:
# Create the lookup table
# For each user, concatenate the committees they are involved in ('Yes' values in committee columns)
complete_data['committees'] = complete_data[committee_columns_no_hanwash].apply(
    lambda row: ', '.join([committee for committee, involvement in row.items() if involvement == 'Yes']),
    axis=1
)

# Select the relevant columns: user_ID, username, and the concatenated committees
lookup_table = complete_data[['user_ID', 'Username', 'committees']]
# Group the data by user_ID and concatenate committees without repetition
# We will group by 'user_ID' and 'Username', and then concatenate the unique committee involvements.

def concatenate_committees(group):
    # For each group (user), concatenate the unique committee names
    committees = ', '.join(sorted(set(', '.join(group['committees']).split(', '))))
    return pd.Series({
        'Username': group['Username'].iloc[0], 
        'committees': committees
    })

# Apply the group by operation and concatenate committees for each unique user
unique_user_committees = lookup_table.groupby('user_ID').apply(concatenate_committees).reset_index()
# Replace empty rows in the 'committees' column with 'HANWASH'
unique_user_committees['committees'] = unique_user_committees['committees'].replace('', 'HANWASH')

unique_user_committees

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_80608/898586387.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_user_committees = lookup_table.groupby('user_ID').apply(concatenate_committees).reset_index()


,user_ID,Username,committees
0,0,AAB_Aquora,HANWASH
1,1,AernoutK,"D7020 Engagement, Steering Committee"
2,2,Alex Bonhomme,", Executive, Monitoring and Evaluation, Steeri..."
3,3,B.Stewart,HANWASH
4,4,Barb S.,HANWASH
...,...,...,...
85,85,tedkirchharr,HANWASH
86,86,wacarlton,HANWASH
87,87,washdominique,"D7020 Engagement, Steering Committee"
88,88,websterlie,"Ambassadors, Steering Committee"


In [14]:
inactive_members_committees = inactive_users[['user_ID','Username']].merge(unique_user_committees[['user_ID','committees']],how='left', on='user_ID' )
active_members_committees =active_users[['user_ID','Username']].merge(unique_user_committees[['user_ID','committees']],how='left', on='user_ID' )

In [15]:
# Ensure that the 'reports' directory exists
output_dir = './reports'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [16]:
past_overall_percentage = (num_active_users_12_months/clean_data.shape[0])*100
past_overall_percentage = round(past_overall_percentage)
past_overall_percentage

58

In [17]:
overall_percentage = (total_active_users/clean_data.shape[0])*100
overall_percentage = round(overall_percentage)
overall_percentage

14

In [18]:
# Now update the report by removing HANWASH from all committee analyses and charts
output_file_excluding_hanwash = os.path.join(output_dir, 'mWater_usage_reports_final_excluding_hanwash.xlsx')

with pd.ExcelWriter(output_file_excluding_hanwash, engine='xlsxwriter') as writer:

    # Write active user analysis to a sheet
    pd.DataFrame({
        'Metric': [
            'Number of Active Users last 12 month',
            'Number of Active Users last 7 days', 
            'Number of Inactive Users last 7 days',
            'Number of Active Portal Users last 7 days', 
            'Number of Active App Users last 7 days',
            'Overall Usage Percentage last 7 days',
            'Overall Usage Percentage last 12 months',
        ],
        'Value': [
            num_active_users_12_months,
            total_active_users, 
            num_inactive_users,
            total_portal_active_users, 
            total_app_active_users, 
            f"{overall_percentage} %",
            f"{past_overall_percentage} %",
        ]
    }).to_excel(writer, sheet_name='Usage Summary', index=False)

    inactive_members_committees.to_excel(writer, sheet_name='inactive_members_committees', index=False)

    active_members_committees.to_excel(writer, sheet_name='active_members_committees', index=False)

    # Write clean_data with deduplication summary
    clean_data.to_excel(writer, sheet_name='Clean Data', index=False)

    """ # Write committee analysis excluding HANWASH
    active_users_by_committee_no_hanwash.to_frame(name='Active Users Count').to_excel(writer, sheet_name='Active Users Committees', index=True)
    inactive_users_by_committee_no_hanwash.to_frame(name='Inactive Users Count').to_excel(writer, sheet_name='Inactive Users Committees', index=True)

    # Insert the updated percentage bar chart excluding HANWASH
    worksheet_percentage_bar_chart_no_hanwash = writer.book.add_worksheet('Percentage Active Excl HANWASH')
    worksheet_percentage_bar_chart_no_hanwash.insert_image('B2', percentage_bar_chart_image_no_hanwash)

    # Insert the overall pie chart for active users vs total users
    worksheet_overall_pie_chart = writer.book.add_worksheet('Active vs Total Pie Chart')
    worksheet_overall_pie_chart.insert_image('B2', overall_pie_chart_image) """

print(f"Final report (excluding HANWASH) generated at: {output_file_excluding_hanwash}")


" # Write committee analysis excluding HANWASH\n    active_users_by_committee_no_hanwash.to_frame(name='Active Users Count').to_excel(writer, sheet_name='Active Users Committees', index=True)\n    inactive_users_by_committee_no_hanwash.to_frame(name='Inactive Users Count').to_excel(writer, sheet_name='Inactive Users Committees', index=True)\n\n    # Insert the updated percentage bar chart excluding HANWASH\n    worksheet_percentage_bar_chart_no_hanwash = writer.book.add_worksheet('Percentage Active Excl HANWASH')\n    worksheet_percentage_bar_chart_no_hanwash.insert_image('B2', percentage_bar_chart_image_no_hanwash)\n\n    # Insert the overall pie chart for active users vs total users\n    worksheet_overall_pie_chart = writer.book.add_worksheet('Active vs Total Pie Chart')\n    worksheet_overall_pie_chart.insert_image('B2', overall_pie_chart_image) "

Final report (excluding HANWASH) generated at: ./reports/mWater_usage_reports_final_excluding_hanwash.xlsx


In [19]:
""" # Create a Bar Chart for the percentage of active users by committee (excluding HANWASH)
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(committee_columns_no_hanwash, active_users_percentage_by_committee_no_hanwash, color='#1f77b4')
ax.set_xlabel('Percentage of Active Users (%)')
ax.set_title('Percentage of Active Users by Committee (Excluding HANWASH)') """

" # Create a Bar Chart for the percentage of active users by committee (excluding HANWASH)\nfig, ax = plt.subplots(figsize=(10, 6))\nax.barh(committee_columns_no_hanwash, active_users_percentage_by_committee_no_hanwash, color='#1f77b4')\nax.set_xlabel('Percentage of Active Users (%)')\nax.set_title('Percentage of Active Users by Committee (Excluding HANWASH)') "

In [20]:
""" 
# Save the updated percentage bar chart (excluding HANWASH)
percentage_bar_chart_image_no_hanwash = os.path.join(output_dir, 'percentage_active_committee_chart_no_hanwash.png')
plt.savefig(percentage_bar_chart_image_no_hanwash, bbox_inches='tight') """

" \n# Save the updated percentage bar chart (excluding HANWASH)\npercentage_bar_chart_image_no_hanwash = os.path.join(output_dir, 'percentage_active_committee_chart_no_hanwash.png')\nplt.savefig(percentage_bar_chart_image_no_hanwash, bbox_inches='tight') "

In [21]:
""" # Calculate total unique users and percentage of active users in the past 12 months over total users
total_unique_users = clean_data.shape[0]
active_users_percentage_overall = (num_active_users_12_months / total_unique_users) * 100

# Create a pie chart showing active users vs. total users
fig, ax = plt.subplots()
ax.pie([num_active_users_12_months, total_unique_users - num_active_users_12_months],
       labels=['Active Users (12 months)', 'Inactive/Other Users'], autopct='%1.1f%%', startangle=90, colors=['#1f77b4', '#ff7f0e'])
ax.set_title('Active Users vs Total Users on Platform')

# Save the overall pie chart
overall_pie_chart_image = os.path.join(output_dir, 'overall_active_users_pie.png')
plt.savefig(overall_pie_chart_image, bbox_inches='tight')
 """

" # Calculate total unique users and percentage of active users in the past 12 months over total users\ntotal_unique_users = clean_data.shape[0]\nactive_users_percentage_overall = (num_active_users_12_months / total_unique_users) * 100\n\n# Create a pie chart showing active users vs. total users\nfig, ax = plt.subplots()\nax.pie([num_active_users_12_months, total_unique_users - num_active_users_12_months],\n       labels=['Active Users (12 months)', 'Inactive/Other Users'], autopct='%1.1f%%', startangle=90, colors=['#1f77b4', '#ff7f0e'])\nax.set_title('Active Users vs Total Users on Platform')\n\n# Save the overall pie chart\noverall_pie_chart_image = os.path.join(output_dir, 'overall_active_users_pie.png')\nplt.savefig(overall_pie_chart_image, bbox_inches='tight')\n "